In [1]:
import argparse, yaml
from openvqa.models.model_loader import CfgLoader
from utils1.exec1 import Execution

In [3]:
import os, copy
import sys
from openvqa.datasets.dataset_loader import DatasetLoader

class Execution:
    def __init__(self, __C):
        self.__C = __C

        print('Loading dataset........')
        self.dataset = DatasetLoader(__C).DataSet()

        # If trigger the evaluation after every epoch
        # Will create a new cfgs with RUN_MODE = 'val'
        self.dataset_eval = None
        if __C.EVAL_EVERY_EPOCH:
            __C_eval = copy.deepcopy(__C)
            setattr(__C_eval, 'RUN_MODE', 'val')

            print('Loading validation set for per-epoch evaluation........')
            self.dataset_eval = DatasetLoader(__C_eval).DataSet()


    def run(self, run_mode):
        
        if run_mode == 'train':
            if self.__C.RESUME is False:
                self.empty_log(self.__C.VERSION)
            train_engine(self.__C, self.dataset, self.dataset_eval)

        elif run_mode == 'val':
            test_engine(self.__C, self.dataset, validation=True)

        elif run_mode == 'test':
            test_engine(self.__C, self.dataset)

        else:
            exit(-1)


    def empty_log(self, version):
        print('Initializing log file........')
        if (os.path.exists(self.__C.LOG_PATH + '/log_run_' + version + '.txt')):
            os.remove(self.__C.LOG_PATH + '/log_run_' + version + '.txt')
        print('Finished!')
        print('')



In [4]:
def parse_args():
    '''
    Parse input arguments
    '''
    parser = argparse.ArgumentParser(description='OpenVQA Args')

    parser.add_argument('--RUN', dest='RUN_MODE',
                      choices=['train', 'val', 'test'],
                      help='{train, val, test}',
                      default='train',
                      type=str, required=False)

    parser.add_argument('--MODEL', dest='MODEL',
                      choices=[
                           'mcan_small',
                           'mcan_small_wa',
                           'mcan_large',
                           'ban_4',
                           #Edits
                           'ban_8_wa',
                           'baseline_wa',
                           #End of Edits
                           'ban_8',
                           'mfb',
                           'mfb_wa',
                           'mfh',
                           'mfh_wa',
                           'mem',
                           'butd',
                           'butd_wa',
                           'baseline',
                           'baseline_wa_no_fusion',
                           'positional',
                           'mcan_large_wa',
                           'mcan_small_augmented',
                           'mcan_small_without_a'
                           ]
                        ,
                      help='{'
                           'mcan_small,'
                           'mcan_small_wa,'
                           'mcan_large,'
                            #Edits
                           'ban_wa,'
                           'baseline_wa,'
                           #End of Edits
                           'ban_4,'
                           'ban_8,'
                           'mfb,'
                           'mfb_wa,'
                           'mfh,'
                           'mfh_wa,'
                           'butd,'
                           'butd_wa,'
                           'baseline,'
                           'baseline_wa_no_fusion,'
                           'positional,'
                           '}'
                        ,
                      type=str, required=True)

    parser.add_argument('--DATASET', dest='DATASET',
                      choices=['vqa', 'gqa', 'clevr'],
                      help='{'
                           'vqa,'
                           'gqa,'
                           'clevr,'
                           '}'
                        ,
                      default='vqa',  
                      type=str, required=False)

    parser.add_argument('--SPLIT', dest='TRAIN_SPLIT',
                      choices=['train', 'train+val', 'train+val+vg'],
                      help="set training split, "
                           "vqa: {'train', 'train+val', 'train+val+vg'}"
                           "gqa: {'train', 'train+val'}"
                           "clevr: {'train', 'train+val'}"
                        ,
                        default='train', required=False,
                      type=str)

    parser.add_argument('--EVAL_EE', dest='EVAL_EVERY_EPOCH',
                      choices=['True', 'False'],
                      help='True: evaluate the val split when an epoch finished,'
                           'False: do not evaluate on local',
                           default='True',
                           required=False,
                      type=str)

    parser.add_argument('--SAVE_PRED', dest='TEST_SAVE_PRED',
                      choices=['True', 'False'],
                      help='True: save the prediction vectors,'
                           'False: do not save the prediction vectors',
                      default='True',
                      required=False,
                      type=str)

    parser.add_argument('--BS', dest='BATCH_SIZE',
                      help='batch size in training',
                      type=int)

    parser.add_argument('--GPU', dest='GPU',
                      help="gpu choose, eg.'0, 1, 2, ...'",
                      default='0, 1',
                      type=str)

    parser.add_argument('--SEED', dest='SEED',
                      help='fix random seed',
                      type=int)

    parser.add_argument('--VERSION', dest='VERSION',
                      help='Enter descriptive name here (eg baseline_wa_gru), will be used for WANDB and for version',
                      required=True,
                      type=str)

    parser.add_argument('--RESUME', dest='RESUME',
                      choices=['True', 'False'],
                      help='True: use checkpoint to resume training,'
                           'False: start training with random init',
                      type=str)

    parser.add_argument('--CKPT_V', dest='CKPT_VERSION',
                      help='checkpoint version',
                      type=str)

    parser.add_argument('--CKPT_E', dest='CKPT_EPOCH',
                      help='checkpoint epoch',
                      type=int)

    parser.add_argument('--CKPT_PATH', dest='CKPT_PATH',
                      help='load checkpoint path, we '
                           'recommend that you use '
                           'CKPT_VERSION and CKPT_EPOCH '
                           'instead, it will override'
                           'CKPT_VERSION and CKPT_EPOCH',
                      type=str)

    parser.add_argument('--ACCU', dest='GRAD_ACCU_STEPS',
                      help='split batch to reduce gpu memory usage',
                      type=int)

    parser.add_argument('--NW', dest='NUM_WORKERS',
                      help='multithreaded loading to accelerate IO',
                      type=int)

    parser.add_argument('--PINM', dest='PIN_MEM',
                      choices=['True', 'False'],
                      help='True: use pin memory, False: not use pin memory',
                      type=str)

    parser.add_argument('--VERB', dest='VERBOSE',
                      choices=['True', 'False'],
                      help='True: verbose print, False: simple print',
                      type=str)

    parser.add_argument('--USE_NEW_QUESTION', dest='USE_NEW_QUESTION',
                      choices=['True', 'False'],
                      help='whether to use new question while testing',
                      default='False',
                      type=str)

    parser.add_argument('--NEW_QUESTION', dest='NEW_QUESTION',
                      help='the new question to be asked while testing',
                      type=str)

    parser.add_argument('--IMAGE_ID', dest='IMAGE_ID',
                      help='image id on which the questions to be asked',
                      type=str)
    
    ######################################################
    #########  CHANGE MODEL AND VERSION HERE #############
    ######################################################
    args = parser.parse_args(args=['--MODEL', 'baseline_wa', '--VERSION', 'hakku'])
    return args

In [5]:
args = parse_args()
print(args)

Namespace(BATCH_SIZE=None, CKPT_EPOCH=None, CKPT_PATH=None, CKPT_VERSION=None, DATASET='vqa', EVAL_EVERY_EPOCH='True', GPU='0, 1', GRAD_ACCU_STEPS=None, IMAGE_ID=None, MODEL='baseline_wa', NEW_QUESTION=None, NUM_WORKERS=None, PIN_MEM=None, RESUME=None, RUN_MODE='train', SEED=None, TEST_SAVE_PRED='True', TRAIN_SPLIT='train', USE_NEW_QUESTION='False', VERBOSE=None, VERSION='hakku')


In [6]:
cfg_file = "configs/{}/{}.yml".format(args.DATASET, args.MODEL)
with open(cfg_file, 'r') as f:

    # Loads the yaml file
    yaml_dict = yaml.load(f)

# Loads the model_cfgs + base_cfgs
__C = CfgLoader(yaml_dict['MODEL_USE']).load()

# Loads the command line cfgs
args = __C.str_to_bool(args)
args_dict = __C.parse_to_dict(args)

# {**dict1, **dict2} creates a new dictionary by merging dict1 and dict2, using dict2 for key clashes
args_dict = {**yaml_dict, **args_dict}
__C.add_args(args_dict)
__C.proc()

# FINAL PREFERENCE OF CFGS:
# COMMAND LINE > YAML FILE > MODEL CFGS > BASE CFGS

print('Hyper Parameters:')
print(__C)

Checking dataset ........
Finished!

Hyper Parameters:
{ ALPHA             }->1
{ ANS_STDDEV        }->0.01
{ AUGMENTED_ANSWER  }->False
{ BATCH_SIZE        }->64
{ BBOXFEAT_EMB_SIZE }->2048
{ BETA              }->30
{ CACHE_PATH        }->./results/cache
{ CAP_DIST          }->0.3
{ CKPTS_PATH        }->./ckpts
{ CKPT_EPOCH        }->0
{ CKPT_PATH         }->None
{ CKPT_VERSION      }->343612
{ DATASET           }->vqa
{ DATA_PATH         }->{'clevr': './data/clevr', 'vqa': './data/vqa', 'gqa': './data/gqa'}
{ DATA_ROOT         }->./data
{ DEVICES           }->[0, 1]
{ DROPOUT_R         }->0.1
{ EVAL_BATCH_SIZE   }->32
{ EVAL_EVERY_EPOCH  }->True
{ FEATS_PATH        }->{'clevr': {'test': './data/clevr/feats/test', 'val': './data/clevr/feats/val', 'train': './data/clevr/feats/train'}, 'vqa': {'test': './data/vqa/feats/test2015', 'val': './data/vqa/feats/val2014', 'train': './data/vqa/feats/train2014'}, 'gqa': {'default-frcn': './data/gqa/feats/gqa-frcn', 'default-grid': './data/gqa/fea

/home/slam/dl/lib/python3.5/site-packages/ipykernel_launcher.py:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """


In [7]:
execution = Execution(__C)

Loading dataset........
Loading all questions (for statistics)
Loading all image features
Loading split questions and answers
 ========== Dataset size: 443757

Tokenising questions
 ========== Question token vocab size: 20573
Tokenising answers
 ========== Answer token vocab size:  13488
 ========== Answer token vocab size (occur more than 8 times): 3129
Finished!

Loading validation set for per-epoch evaluation........
Loading all questions (for statistics)
Loading all image features
Loading split questions and answers
 ========== Dataset size: 214354

Tokenising questions
 ========== Question token vocab size: 20573
 ========== Answer token vocab size (occur more than 8 times): 3129
Finished!



In [ ]:
import os, torch, datetime, shutil, time
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as Data
import wandb
from openvqa.models.model_loader import ModelLoader
from openvqa.utils.optim import get_optim, adjust_lr
from utils1.test_engine import test_engine, ckpt_proc
from vis import plotter, vis_func
from multiprocessing import Pool
import multiprocessing
import sys
from torchviz import make_dot, make_dot_from_trace

def train_engine(__C, dataset, dataset_eval=None):

    data_size = dataset.data_size
    token_size = dataset.token_size
    ans_size = dataset.ans_size
    pretrained_emb = dataset.pretrained_emb

    #Edits
    pretrained_emb_ans = dataset.pretrained_emb_ans
    token_size_ans = dataset.token_size_ans #End of Edits

    print("Model being used is {}".format(__C.MODEL_USE))

    net = ModelLoader(__C).Net(
        __C,
        pretrained_emb,
        token_size,
        ans_size,
        pretrained_emb_ans,
        token_size_ans
    )

    net.cuda()
    net.train()

    if __C.N_GPU > 1:
        net = nn.DataParallel(net, device_ids=__C.DEVICES)

    # Define Loss Function
    loss_fn = eval('torch.nn.' + __C.LOSS_FUNC_NAME_DICT[__C.LOSS_FUNC] + "(reduction='" + __C.LOSS_REDUCTION + "').cuda()")


    # creating a folder for saving the numpy visualization arrays
    if (__C.WITH_ANSWER and ((__C.VERSION) not in os.listdir(__C.SAVED_PATH))):
        os.mkdir(__C.SAVED_PATH + '/' + __C.VERSION)


    # Load checkpoint if resume training
    if __C.RESUME:
        print(' ========== Resume training')

        if __C.CKPT_PATH is not None:
            print('Warning: Now using CKPT_PATH args, '
                  'CKPT_VERSION and CKPT_EPOCH will not work')
            path = __C.CKPT_PATH
        else:
            path = __C.CKPTS_PATH + \
                   '/ckpt_' + __C.CKPT_VERSION + \
                   '/epoch' + str(__C.CKPT_EPOCH) + '.pkl'

        # Load the network parameters
        print('Loading ckpt from {}'.format(path))
        ckpt = torch.load(path)
        print('Finish!')

        if __C.N_GPU > 1:
            net.load_state_dict(ckpt_proc(ckpt['state_dict']))
        else:
            net.load_state_dict(ckpt['state_dict'])
        start_epoch = ckpt['epoch']

        # Load the optimizer paramters
        optim = get_optim(__C, net, data_size, ckpt['lr_base'])
        optim._step = int(data_size / __C.BATCH_SIZE * start_epoch)
        optim.optimizer.load_state_dict(ckpt['optimizer'])
        
        if ('ckpt_' + __C.VERSION) not in os.listdir(__C.CKPTS_PATH):
            os.mkdir(__C.CKPTS_PATH + '/ckpt_' + __C.VERSION)

    else:
        if ('ckpt_' + __C.VERSION) not in os.listdir(__C.CKPTS_PATH):
            #shutil.rmtree(__C.CKPTS_PATH + '/ckpt_' + __C.VERSION)
            os.mkdir(__C.CKPTS_PATH + '/ckpt_' + __C.VERSION)

        optim = get_optim(__C, net, data_size)
        start_epoch = 0

    loss_sum = 0
    named_params = list(net.named_parameters())
    grad_norm = np.zeros(len(named_params))

    # Define multi-thread dataloader
    # if __C.SHUFFLE_MODE in ['external']:
    #     dataloader = Data.DataLoader(
    #         dataset,
    #         batch_size=__C.BATCH_SIZE,
    #         shuffle=False,
    #         num_workers=__C.NUM_WORKERS,
    #         pin_memory=__C.PIN_MEM,
    #         drop_last=True
    #     )
    # else:
    dataloader = Data.DataLoader(
        dataset,
        batch_size=__C.BATCH_SIZE,
        shuffle=True,
        num_workers=__C.NUM_WORKERS,
        pin_memory=__C.PIN_MEM,
        drop_last=True
    )

    logfile = open(
        __C.LOG_PATH +
        '/log_run_' + __C.VERSION + '.txt',
        'a+'
    )
    logfile.write(str(__C))
    logfile.close()

    # For dry runs
    # os.environ['WANDB_MODE'] = 'dryrun' 

    # initializing the wandb project
    # TODO to change the name of project later, once the proper coding starts
    #wandb.init(project="openvqa", name=__C.VERSION, config=__C)

    # obtain histogram of each gradients in network as it trains
    #wandb.watch(net, log="all")

    #wandb.save("./openvqa/models/" + str(__C.MODEL_USE) + "/net.py")
    #wandb.save("./utils1/train_engine.py")

    # Training script
    for epoch in range(start_epoch, __C.MAX_EPOCH):

        # Save log to file
        logfile = open(
            __C.LOG_PATH +
            '/log_run_' + __C.VERSION + '.txt',
            'a+'
        )
        logfile.write(
            '=====================================\nnowTime: ' +
            datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S') +
            '\n'
        )
        logfile.close()

        # Learning Rate Decay
        if epoch in __C.LR_DECAY_LIST:
            adjust_lr(optim, __C.LR_DECAY_R)

        # Externally shuffle data list
        # if __C.SHUFFLE_MODE == 'external':
        #     dataset.shuffle_list(dataset.ans_list)

        time_start = time.time()
        # Iteration
        for step, (
                frcn_feat_iter,
                grid_feat_iter,
                bbox_feat_iter,
                ques_ix_iter,

                #Edits
                ans_ix_iter,
                #End of Edits

                ans_iter,
                ques_type

        ) in enumerate(dataloader):

            optim.zero_grad()

            frcn_feat_iter = frcn_feat_iter.cuda()
            grid_feat_iter = grid_feat_iter.cuda()
            bbox_feat_iter = bbox_feat_iter.cuda()
            ques_ix_iter = ques_ix_iter.cuda()
            #Edits
            ans_ix_iter = ans_ix_iter.cuda()
            #End of Edits
            ans_iter = ans_iter.cuda()

            loss_tmp = 0

            loss_img_ques_tmp = 0
            loss_ans_tmp = 0
            loss_interp_tmp = 0
            loss_fusion_tmp = 0

            for accu_step in range(__C.GRAD_ACCU_STEPS):
                loss_tmp = 0
                loss_img_ques_tmp = 0
                loss_ans_tmp = 0
                loss_interp_tmp = 0
                loss_fusion_tmp = 0

                sub_frcn_feat_iter = \
                    frcn_feat_iter[accu_step * __C.SUB_BATCH_SIZE:
                                  (accu_step + 1) * __C.SUB_BATCH_SIZE]
                sub_grid_feat_iter = \
                    grid_feat_iter[accu_step * __C.SUB_BATCH_SIZE:
                                  (accu_step + 1) * __C.SUB_BATCH_SIZE]
                sub_bbox_feat_iter = \
                    bbox_feat_iter[accu_step * __C.SUB_BATCH_SIZE:
                                  (accu_step + 1) * __C.SUB_BATCH_SIZE]
                sub_ques_ix_iter = \
                    ques_ix_iter[accu_step * __C.SUB_BATCH_SIZE:
                                 (accu_step + 1) * __C.SUB_BATCH_SIZE]
                #Edits
                sub_ans_ix_iter = \
                    ans_ix_iter[accu_step * __C.SUB_BATCH_SIZE:
                                 (accu_step + 1) * __C.SUB_BATCH_SIZE]
                #End of Edits

                sub_ans_iter = \
                    ans_iter[accu_step * __C.SUB_BATCH_SIZE:
                             (accu_step + 1) * __C.SUB_BATCH_SIZE]

                
                # when making predictions also pass the ans_iter which is a dictionary from which you
                # can extract answers and pass them through decoders

                if (__C.WITH_ANSWER):
                    pred_img_ques, pred_ans, pred_fused, z_img_ques, z_ans, z_fused = net(
                        sub_frcn_feat_iter,
                        sub_grid_feat_iter,
                        sub_bbox_feat_iter,
                        sub_ques_ix_iter,
                        sub_ans_ix_iter,
                        step,
                        epoch
                    )
                else:
                     pred_img_ques = net(
                        sub_frcn_feat_iter,
                        sub_grid_feat_iter,
                        sub_bbox_feat_iter,
                        sub_ques_ix_iter,
                        sub_ans_ix_iter,
                        step,
                        epoch
                    )


                # we need to change the loss terms accordingly
                # now we need to modify the loss terms for the same
                
                #Edits: creating the loss items for each of the prediction vector

                loss_item_img_ques = [pred_img_ques, sub_ans_iter]

                # only calculate the ans and interp loss in case of WITH_ANSWER
                if (__C.WITH_ANSWER):
                    loss_item_ans = [pred_ans, sub_ans_iter]
                    loss_item_interp = [pred_fused, sub_ans_iter]

                
                loss_nonlinear_list = __C.LOSS_FUNC_NONLINEAR[__C.LOSS_FUNC]
                
                # applying the same transformation on the all three
                # althought for 'bce' loss the following does nothing
                for item_ix, loss_nonlinear in enumerate(loss_nonlinear_list):
                    if loss_nonlinear in ['flat']:
                        loss_item_img_ques[item_ix] = loss_item_img_ques[item_ix].view(-1)
                    elif loss_nonlinear:
                        loss_item_img_ques[item_ix] = eval('F.' + loss_nonlinear + '(loss_item_img_ques[item_ix], dim=1)')

                for item_ix, loss_nonlinear in enumerate(loss_nonlinear_list):
                    if loss_nonlinear in ['flat'] and __C.WITH_ANSWER:
                        loss_item_ans[item_ix] = loss_item_ans[item_ix].view(-1)
                    elif loss_nonlinear and __C.WITH_ANSWER:
                        loss_item_ans[item_ix] = eval('F.' + loss_nonlinear + '(loss_item_ans[item_ix], dim=1)')

                for item_ix, loss_nonlinear in enumerate(loss_nonlinear_list):
                    if loss_nonlinear in ['flat'] and __C.WITH_ANSWER:
                        loss_item_interp[item_ix] = loss_item_interp[item_ix].view(-1)
                    elif loss_nonlinear and __C.WITH_ANSWER:
                        loss_item_interp[item_ix] = eval('F.' + loss_nonlinear + '(loss_item_interp[item_ix], dim=1)')


                # Now we create all the four losses and then add them
                #print("shape of loss_item_img_ques[0] is {} and of loss_item_img_ques[1] is {}".format(loss_item_img_ques[0],loss_item_img_ques[1]))
                loss_img_ques = loss_fn(loss_item_img_ques[0], loss_item_img_ques[1])


                loss = 0
                loss += loss_img_ques
                
                if (__C.WITH_ANSWER):

                    # loss for the prediction from the answer
                    #print("shape of loss_item_ans[0] is {} and of loss_item_ans[1] is {}".format(loss_item_ans[0],loss_item_ans[1]))
                    loss_ans = loss_fn(loss_item_ans[0], loss_item_ans[1])
                
                    # Loss for the prediction from the fused vector
                    # I am keeping the loss same as bce but we can change it later for more predictions
                    # loss_fused = interpolation loss
                    #print("shape of loss_item_interp[0] is {} and of loss_item_interp[1] is {}".format(loss_item_interp[0],loss_item_interp[1]))
                    loss_interp = loss_fn(loss_item_interp[0], loss_item_interp[1])
                    
                    # we also need to multiply this fused loss by a hyperparameter alpha
                    # put the alpha in the config and uncomment the following line
                    loss_interp *= __C.ALPHA
                    loss += loss_ans + loss_interp

                    print("\n----------  FIRST LOSS  --------")
                    optim.zero_grad()
                    loss.backward(retain_graph=True)
                    params1 = list(net.parameters())
                    '''
                    lays = 0
                    for name, i in net.named_parameters():
                        print(name, end = ' ')
                        if (i.requires_grad == False):
                            print ("WWWWWWWWWWWWHHHHHHHHHHHAAAAAAAAAAAAAATTTTTTTTTTTTTTTTTTTT")
                        if ((i.grad != 0).sum() == 0):
                            print(' NOT UPDATING')
                            lays += 1
                        else:
                            print(' ')
                    print ("Gradient not updating in: ", lays, ' of total: ', len(params1))
                    '''



                    if (__C.WITH_FUSION_LOSS):

                        # Now calculate the fusion loss
                        #1. Higher loss for higher distance between vectors predicted
                        # by different models for same example

                        dist_calc = (z_img_ques - z_ans).pow(2).sum(1).sqrt()
                        #print("Count of distances being clipped (true is clipped): ", np.unique((dist_calc > __C.CAP_DIST).cpu().numpy(), return_counts=True))

                        '''
                        loss_fusion = torch.min(
                                torch.tensor(__C.CAP_DIST).cuda(),
                                dist_calc
                                ).mean()

                        #2. Lower loss for more distance between two pred vectors of same model
                        loss_fusion -= torch.min(
                                torch.tensor(__C.CAP_DIST).cuda(), 
                                torch.pdist(z_img_ques, 2)
                                ).mean() 

                        loss_fusion -= torch.min(
                                torch.tensor(__C.CAP_DIST).cuda(), 
                                torch.pdist(z_ans, 2)
                                ).mean() 
                        '''

                        loss_fusion = dist_calc.mean()

                        #2. Lower loss for more distance between two pred vectors of same model
                        '''
                        calculating pairwise intra distance on same type questions
                        '''
                        '''
                        types = ['other', 'yes/no', 'number']
                        for i in range(3):
                            j = (i+1)%3
                            indices_i = [k for k, val in enumerate(ques_type) if val == types[i]]
                            indices_j = [k for k, val in enumerate(ques_type) if val == types[j]]
                            if ((indices_i != []) and (indices_j != [])):
                                loss_fusion -= torch.cdist(z_img_ques[indices_i], z_img_ques[indices_j]).mean()
                                loss_fusion -= torch.cdist(z_ans[indices_i], z_ans[indices_j]).mean()
                            if (indices_i != []):
                                loss_fusion += torch.pdist(z_img_ques[indices_i], 2).mean()
                                loss_fusion += torch.pdist(z_ans[indices_i], 2).mean()
                        '''
                        loss_fusion -= torch.pdist(z_img_ques, 2).mean() 

                        loss_fusion -= torch.pdist(z_ans, 2).mean() 


                        # Multiply the loss fusion with hyperparameter beta
                        loss_fusion *= __C.BETA

                        #print('fusion loss is : {}'.format(loss_fusion))

                        print("\n----------  FUSION LOSS  --------")
                        optim.zero_grad()
                        loss_fusion.backward(retain_graph=True)
                        params2 = list(net.parameters())
                        lays = 0
                        dot_sum = 0
                        for idx, (name, i) in enumerate(net.named_parameters()):
                            dot_sum += (params1[idx] * params2[idx]).sum()
                            print(name, ' dot: ', (params1[idx].grad * i.grad).sum().item(), end = '   ')
                            if ((i.grad != 0).sum() == 0):
                                print(' NOT UPDATING')
                                lays += 1
                            else:
                                print(' ')
                        print ("Gradient not updating in: ", lays, ' of total: ', len(params2))
                        print("@@@@@@@@@@@@@@@@@@@ Overall dot product: ", dot_sum.item())

                        loss += loss_fusion

                loss /= __C.GRAD_ACCU_STEPS

                print("\n----------  MAIN LOSS  --------")
                optim.zero_grad()
                loss.backward()
                params = list(net.parameters())
                '''
                lays = 0
                for name, i in net.named_parameters():
                    print(name, end = ' ')
                    if (i.requires_grad == False):
                        print ("WWWWWWWWWWWWHHHHHHHHHHHAAAAAAAAAAAAAATTTTTTTTTTTTTTTTTTTT")
                    if ((i.grad != 0).sum() == 0):
                        print(' NOT UPDATING')
                        lays += 1
                    else:
                        print(' ')
                print ("Gradient not updating in: ", lays, ' of total: ', len(params))
                '''

                loss_tmp += loss.cpu().data.numpy() * __C.GRAD_ACCU_STEPS
                loss_sum += loss.cpu().data.numpy() * __C.GRAD_ACCU_STEPS

                # calculating temp loss of each type
                if __C.WITH_ANSWER:
                    loss_img_ques_tmp += loss_img_ques.cpu().data.numpy() * __C.GRAD_ACCU_STEPS
                    loss_ans_tmp += loss_ans.cpu().data.numpy() * __C.GRAD_ACCU_STEPS
                    loss_interp_tmp += loss_interp.cpu().data.numpy() * __C.GRAD_ACCU_STEPS
                    if (__C.WITH_FUSION_LOSS):
                        loss_fusion_tmp += loss_fusion.cpu().data.numpy() * __C.GRAD_ACCU_STEPS


            if __C.VERBOSE:
                if dataset_eval is not None:
                    mode_str = __C.SPLIT['train'] + '->' + __C.SPLIT['val']
                else:
                    mode_str = __C.SPLIT['train'] + '->' + __C.SPLIT['test']

                print("\r[Version %s][Epoch %2d][Step %4d/%4d] Loss: %.4f [iq: %.4f,ans: %.4f,interp: %.4f,fusion: %.4f]" % (
                    __C.VERSION,
                    epoch + 1,
                    step,
                    int(data_size / __C.BATCH_SIZE),
                    loss_tmp / __C.SUB_BATCH_SIZE,
                    loss_img_ques_tmp / __C.SUB_BATCH_SIZE,
                    loss_ans_tmp / __C.SUB_BATCH_SIZE,
                    loss_interp_tmp / __C.SUB_BATCH_SIZE,
                    loss_fusion_tmp / __C.SUB_BATCH_SIZE
                ), end = '          ')

            # Gradient norm clipping
            if __C.GRAD_NORM_CLIP > 0:
                nn.utils.clip_grad_norm_(
                    net.parameters(),
                    __C.GRAD_NORM_CLIP
                )

            # Save the gradient information
            for name in range(len(named_params)):
                norm_v = torch.norm(named_params[name][1].grad).cpu().data.numpy() \
                    if named_params[name][1].grad is not None else 0
                grad_norm[name] += norm_v * __C.GRAD_ACCU_STEPS
                # print('Param %-3s Name %-80s Grad_Norm %-20s'%
                #       (str(grad_wt),
                #        params[grad_wt][0],
                #        str(norm_v)))

            optim.step()

        time_end = time.time()
        elapse_time = time_end-time_start
        print('Finished in {}s'.format(int(elapse_time)))
        epoch_finish = epoch + 1

        # Save checkpoint
        if __C.N_GPU > 1:
            state = {
                'state_dict': net.module.state_dict(),
                'optimizer': optim.optimizer.state_dict(),
                'lr_base': optim.lr_base,
                'epoch': epoch_finish
            }
        else:
            state = {
                'state_dict': net.state_dict(),
                'optimizer': optim.optimizer.state_dict(),
                'lr_base': optim.lr_base,
                'epoch': epoch_finish
            }
        torch.save(
            state,
            __C.CKPTS_PATH +
            '/ckpt_' + __C.VERSION +
            '/epoch' + str(epoch_finish) +
            '.pkl'
        )

        '''
        wandb.save(
            __C.CKPTS_PATH +
            '/ckpt_' + __C.VERSION +
            '/epoch' + str(epoch_finish) +
            '.h5'
        )
        '''
        
        # Logging
        logfile = open(
            __C.LOG_PATH +
            '/log_run_' + __C.VERSION + '.txt',
            'a+'
        )
        logfile.write(
            'Epoch: ' + str(epoch_finish) +
            ', Loss: ' + str(loss_sum / data_size) +
            ', Lr: ' + str(optim._rate) + '\n' +
            'Elapsed time: ' + str(int(elapse_time)) + 
            ', Speed(s/batch): ' + str(elapse_time / step) +
            '\n\n'
        )
        logfile.close()

        '''
        wandb.log({
            'Loss': float(loss_sum / data_size),
            'Learning Rate': optim._rate,
            'Elapsed time': int(elapse_time) 
            })
        '''

        # ---------------------------------------------- #
        # ---- Create visualizations in new processes----#
        # ---------------------------------------------- #
        dic = {}
        dic['version'] = __C.VERSION
        dic['epoch'] = epoch 
        dic['num_samples'] = 1000

        p = Pool(processes= 1)
        p.map_async(vis_func, (dic, ))
        p.close()

        # Eval after every epoch
        epoch_dict = {
                'current_epoch': epoch
                }
        __C.add_args(epoch_dict)
        if dataset_eval is not None:
            test_engine(
                __C,
                dataset_eval,
                state_dict=net.state_dict(),
                validation=True,
                epoch = 0
            )
        p.join()

        # if self.__C.VERBOSE:
        #     logfile = open(
        #         self.__C.LOG_PATH +
        #         '/log_run_' + self.__C.VERSION + '.txt',
        #         'a+'
        #     )
        #     for name in range(len(named_params)):
        #         logfile.write(
        #             'Param %-3s Name %-80s Grad_Norm %-25s\n' % (
        #                 str(name),
        #                 named_params[name][0],
        #                 str(grad_norm[name] / data_size * self.__C.BATCH_SIZE)
        #             )
        #         )
        #     logfile.write('\n')
        #     logfile.close()

        loss_sum = 0
        grad_norm = np.zeros(len(named_params))



In [ ]:
execution.run(__C.RUN_MODE)

Initializing log file........
Finished!

Model being used is baseline_wa

----------  FIRST LOSS  --------

----------  FUSION LOSS  --------
module.decoder_mlp_1.fc.linear.weight  dot:  0.0    NOT UPDATING
module.decoder_mlp_1.fc.linear.bias  dot:  0.0    NOT UPDATING
module.decoder_mlp_1.linear.weight  dot:  0.0    NOT UPDATING
module.decoder_mlp_1.linear.bias  dot:  0.0    NOT UPDATING
module.decoder_mlp_2.fc.linear.weight  dot:  0.0    NOT UPDATING
module.decoder_mlp_2.fc.linear.bias  dot:  0.0    NOT UPDATING
module.decoder_mlp_2.linear.weight  dot:  0.0    NOT UPDATING
module.decoder_mlp_2.linear.bias  dot:  0.0    NOT UPDATING
module.embedding.weight  dot:  2467.458740234375    
module.ans_embedding.weight  dot:  1037403.5    
module.lstm.weight_ih_l0  dot:  3137748.5    
module.lstm.weight_hh_l0  dot:  127333.890625    
module.lstm.bias_ih_l0  dot:  264328.875    
module.lstm.bias_hh_l0  dot:  264328.875    
module.ans_lstm.weight_ih_l0  dot:  169729856.0    
module.ans_lstm.we

[Version hakku][Epoch  1][Step    3/6933] Loss: 6225.2241 [iq: 2162.2109,ans: 2167.1626,interp: 2163.0522,fusion: -267.2014]          
----------  FIRST LOSS  --------

----------  FUSION LOSS  --------
module.decoder_mlp_1.fc.linear.weight  dot:  0.0    NOT UPDATING
module.decoder_mlp_1.fc.linear.bias  dot:  0.0    NOT UPDATING
module.decoder_mlp_1.linear.weight  dot:  0.0    NOT UPDATING
module.decoder_mlp_1.linear.bias  dot:  0.0    NOT UPDATING
module.decoder_mlp_2.fc.linear.weight  dot:  0.0    NOT UPDATING
module.decoder_mlp_2.fc.linear.bias  dot:  0.0    NOT UPDATING
module.decoder_mlp_2.linear.weight  dot:  0.0    NOT UPDATING
module.decoder_mlp_2.linear.bias  dot:  0.0    NOT UPDATING
module.embedding.weight  dot:  2848.955810546875    
module.ans_embedding.weight  dot:  1162367.75    
module.lstm.weight_ih_l0  dot:  3254530.0    
module.lstm.weight_hh_l0  dot:  121089.1875    
module.lstm.bias_ih_l0  dot:  268234.0    
module.lstm.bias_hh_l0  dot:  268234.0    
module.ans_lst

[Version hakku][Epoch  1][Step    7/6933] Loss: 6121.9053 [iq: 2153.7622,ans: 2164.7441,interp: 2159.1809,fusion: -355.7815]          
----------  FIRST LOSS  --------

----------  FUSION LOSS  --------
module.decoder_mlp_1.fc.linear.weight  dot:  0.0    NOT UPDATING
module.decoder_mlp_1.fc.linear.bias  dot:  0.0    NOT UPDATING
module.decoder_mlp_1.linear.weight  dot:  0.0    NOT UPDATING
module.decoder_mlp_1.linear.bias  dot:  0.0    NOT UPDATING
module.decoder_mlp_2.fc.linear.weight  dot:  0.0    NOT UPDATING
module.decoder_mlp_2.fc.linear.bias  dot:  0.0    NOT UPDATING
module.decoder_mlp_2.linear.weight  dot:  0.0    NOT UPDATING
module.decoder_mlp_2.linear.bias  dot:  0.0    NOT UPDATING
module.embedding.weight  dot:  2608.907958984375    
module.ans_embedding.weight  dot:  923786.875    
module.lstm.weight_ih_l0  dot:  2856563.5    
module.lstm.weight_hh_l0  dot:  113841.0625    
module.lstm.bias_ih_l0  dot:  235744.703125    
module.lstm.bias_hh_l0  dot:  235744.703125    
modu

module.attflat_ans.linear_merge.weight  dot:  119186360.0    
module.attflat_ans.linear_merge.bias  dot:  364977792.0    
module.proj_norm.a_2  dot:  0.0    NOT UPDATING
module.proj_norm.b_2  dot:  0.0    NOT UPDATING
module.proj.weight  dot:  0.0    NOT UPDATING
module.proj.bias  dot:  0.0    NOT UPDATING
module.ans_proj_norm.a_2  dot:  0.0    NOT UPDATING
module.ans_proj_norm.b_2  dot:  0.0    NOT UPDATING
module.ans_proj.weight  dot:  0.0    NOT UPDATING
module.ans_proj.bias  dot:  0.0    NOT UPDATING
Gradient not updating in:  16  of total:  46
@@@@@@@@@@@@@@@@@@@ Overall dot product:  1452232.625

----------  MAIN LOSS  --------
[Version hakku][Epoch  1][Step   11/6933] Loss: 6088.2695 [iq: 2142.7622,ans: 2161.8909,interp: 2159.2998,fusion: -375.6831]          
----------  FIRST LOSS  --------

----------  FUSION LOSS  --------
module.decoder_mlp_1.fc.linear.weight  dot:  0.0    NOT UPDATING
module.decoder_mlp_1.fc.linear.bias  dot:  0.0    NOT UPDATING
module.decoder_mlp_1.linear

module.proj.bias  dot:  0.0    NOT UPDATING
module.ans_proj_norm.a_2  dot:  0.0    NOT UPDATING
module.ans_proj_norm.b_2  dot:  0.0    NOT UPDATING
module.ans_proj.weight  dot:  0.0    NOT UPDATING
module.ans_proj.bias  dot:  0.0    NOT UPDATING
Gradient not updating in:  16  of total:  46
@@@@@@@@@@@@@@@@@@@ Overall dot product:  1452234.375

----------  MAIN LOSS  --------
[Version hakku][Epoch  1][Step   15/6933] Loss: 6002.6875 [iq: 2126.3984,ans: 2158.1035,interp: 2137.9878,fusion: -419.8025]          
----------  FIRST LOSS  --------

----------  FUSION LOSS  --------
module.decoder_mlp_1.fc.linear.weight  dot:  0.0    NOT UPDATING
module.decoder_mlp_1.fc.linear.bias  dot:  0.0    NOT UPDATING
module.decoder_mlp_1.linear.weight  dot:  0.0    NOT UPDATING
module.decoder_mlp_1.linear.bias  dot:  0.0    NOT UPDATING
module.decoder_mlp_2.fc.linear.weight  dot:  0.0    NOT UPDATING
module.decoder_mlp_2.fc.linear.bias  dot:  0.0    NOT UPDATING
module.decoder_mlp_2.linear.weight  dot: 

...

[Version hakku][Epoch  1][Step 1028/6933] Loss: -927.6677 [iq: 7.3332,ans: 6.6845,interp: 6.8615,fusion: -948.5469]          
----------  FIRST LOSS  --------

----------  FUSION LOSS  --------
module.decoder_mlp_1.fc.linear.weight  dot:  0.0    NOT UPDATING
module.decoder_mlp_1.fc.linear.bias  dot:  0.0    NOT UPDATING
module.decoder_mlp_1.linear.weight  dot:  0.0    NOT UPDATING
module.decoder_mlp_1.linear.bias  dot:  0.0    NOT UPDATING
module.decoder_mlp_2.fc.linear.weight  dot:  0.0    NOT UPDATING
module.decoder_mlp_2.fc.linear.bias  dot:  0.0    NOT UPDATING
module.decoder_mlp_2.linear.weight  dot:  0.0    NOT UPDATING
module.decoder_mlp_2.linear.bias  dot:  0.0    NOT UPDATING
module.embedding.weight  dot:  178023.453125    
module.ans_embedding.weight  dot:  744475.9375    
module.lstm.weight_ih_l0  dot:  4204976.0    
module.lstm.weight_hh_l0  dot:  714282.3125    
module.lstm.bias_ih_l0  dot:  25794.490234375    
module.lstm.bias_hh_l0  dot:  25794.490234375    
module.ans_l

[Version hakku][Epoch  1][Step 1032/6933] Loss: -903.6978 [iq: 8.3874,ans: 7.7181,interp: 7.5938,fusion: -927.3971]          
----------  FIRST LOSS  --------

----------  FUSION LOSS  --------
module.decoder_mlp_1.fc.linear.weight  dot:  0.0    NOT UPDATING
module.decoder_mlp_1.fc.linear.bias  dot:  0.0    NOT UPDATING
module.decoder_mlp_1.linear.weight  dot:  0.0    NOT UPDATING
module.decoder_mlp_1.linear.bias  dot:  0.0    NOT UPDATING
module.decoder_mlp_2.fc.linear.weight  dot:  0.0    NOT UPDATING
module.decoder_mlp_2.fc.linear.bias  dot:  0.0    NOT UPDATING
module.decoder_mlp_2.linear.weight  dot:  0.0    NOT UPDATING
module.decoder_mlp_2.linear.bias  dot:  0.0    NOT UPDATING
module.embedding.weight  dot:  167420.25    
module.ans_embedding.weight  dot:  1104889.75    
module.lstm.weight_ih_l0  dot:  3581422.5    
module.lstm.weight_hh_l0  dot:  3024548.5    
module.lstm.bias_ih_l0  dot:  245135.953125    
module.lstm.bias_hh_l0  dot:  245135.953125    
module.ans_lstm.weight_

[Version hakku][Epoch  1][Step 1036/6933] Loss: -899.0144 [iq: 7.8957,ans: 7.5676,interp: 8.1081,fusion: -922.5858]          
----------  FIRST LOSS  --------

----------  FUSION LOSS  --------
module.decoder_mlp_1.fc.linear.weight  dot:  0.0    NOT UPDATING
module.decoder_mlp_1.fc.linear.bias  dot:  0.0    NOT UPDATING
module.decoder_mlp_1.linear.weight  dot:  0.0    NOT UPDATING
module.decoder_mlp_1.linear.bias  dot:  0.0    NOT UPDATING
module.decoder_mlp_2.fc.linear.weight  dot:  0.0    NOT UPDATING
module.decoder_mlp_2.fc.linear.bias  dot:  0.0    NOT UPDATING
module.decoder_mlp_2.linear.weight  dot:  0.0    NOT UPDATING
module.decoder_mlp_2.linear.bias  dot:  0.0    NOT UPDATING
module.embedding.weight  dot:  322317.625    
module.ans_embedding.weight  dot:  437679.96875    
module.lstm.weight_ih_l0  dot:  4909859.5    
module.lstm.weight_hh_l0  dot:  8552814.0    
module.lstm.bias_ih_l0  dot:  539463.0625    
module.lstm.bias_hh_l0  dot:  539463.0625    
module.ans_lstm.weight_i

[Version hakku][Epoch  1][Step 1040/6933] Loss: -894.9489 [iq: 9.4573,ans: 8.6675,interp: 8.9594,fusion: -922.0330]          
----------  FIRST LOSS  --------

----------  FUSION LOSS  --------
module.decoder_mlp_1.fc.linear.weight  dot:  0.0    NOT UPDATING
module.decoder_mlp_1.fc.linear.bias  dot:  0.0    NOT UPDATING
module.decoder_mlp_1.linear.weight  dot:  0.0    NOT UPDATING
module.decoder_mlp_1.linear.bias  dot:  0.0    NOT UPDATING
module.decoder_mlp_2.fc.linear.weight  dot:  0.0    NOT UPDATING
module.decoder_mlp_2.fc.linear.bias  dot:  0.0    NOT UPDATING
module.decoder_mlp_2.linear.weight  dot:  0.0    NOT UPDATING
module.decoder_mlp_2.linear.bias  dot:  0.0    NOT UPDATING
module.embedding.weight  dot:  2462854.75    
module.ans_embedding.weight  dot:  1082109.75    
module.lstm.weight_ih_l0  dot:  28792640.0    
module.lstm.weight_hh_l0  dot:  23619810.0    
module.lstm.bias_ih_l0  dot:  2425760.0    
module.lstm.bias_hh_l0  dot:  2425760.0    
module.ans_lstm.weight_ih_l0

[Version hakku][Epoch  1][Step 1044/6933] Loss: -956.4188 [iq: 7.1945,ans: 7.2673,interp: 7.4817,fusion: -978.3622]          
----------  FIRST LOSS  --------

----------  FUSION LOSS  --------
module.decoder_mlp_1.fc.linear.weight  dot:  0.0    NOT UPDATING
module.decoder_mlp_1.fc.linear.bias  dot:  0.0    NOT UPDATING
module.decoder_mlp_1.linear.weight  dot:  0.0    NOT UPDATING
module.decoder_mlp_1.linear.bias  dot:  0.0    NOT UPDATING
module.decoder_mlp_2.fc.linear.weight  dot:  0.0    NOT UPDATING
module.decoder_mlp_2.fc.linear.bias  dot:  0.0    NOT UPDATING
module.decoder_mlp_2.linear.weight  dot:  0.0    NOT UPDATING
module.decoder_mlp_2.linear.bias  dot:  0.0    NOT UPDATING
module.embedding.weight  dot:  1032890.375    
module.ans_embedding.weight  dot:  320127.0625    
module.lstm.weight_ih_l0  dot:  28736028.0    
module.lstm.weight_hh_l0  dot:  8562015.0    
module.lstm.bias_ih_l0  dot:  1868976.0    
module.lstm.bias_hh_l0  dot:  1868976.0    
module.ans_lstm.weight_ih_l